# Distributed model inference using TensorFlow Keras
From: https://docs.databricks.com/_static/notebooks/deep-learning/keras-metadata.html

In [2]:
import os
import shutil
import subprocess
import time
import pandas as pd
from PIL import Image
import numpy as np
import uuid
 
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
 
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql import SparkSession

2024-09-20 15:57:35.101927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 15:57:35.101964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 15:57:35.103443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 15:57:35.111326: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 15:57:35.858194: W tensorflow/comp

In [3]:
num_threads = 6

# Creating a local Spark session for demonstration, in case it hasn't already been created.

_config = {
    "spark.master": f"local[{num_threads}]",
    "spark.driver.host": "127.0.0.1",
    "spark.task.maxFailures": "1",
    "spark.driver.memory": "8g",
    "spark.executor.memory": "8g",
    "spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled": "false",
    "spark.sql.pyspark.jvmStacktrace.enabled": "true",
    "spark.sql.execution.arrow.pyspark.enabled": "true",
    "spark.python.worker.reuse": "true",
}
spark = SparkSession.builder.appName("spark-dl-example")
for key, value in _config.items():
    spark = spark.config(key, value)
spark = spark.getOrCreate()

sc = spark.sparkContext

24/09/20 15:57:37 WARN Utils: Your hostname, dgx2h0194.spark.sjc4.nvmetal.net resolves to a loopback address: 127.0.1.1; using 10.150.30.2 instead (on interface enp134s0f0np0)
24/09/20 15:57:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 15:57:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/20 15:57:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
#import os
#os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda" # set to path containing /cuda/nvvm/libdevice/

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [5]:
file_name = "image_data.parquet"
output_file_path = "predictions"

### Prepare trained model and data for inference

Load the ResNet-50 Model and broadcast the weights.

In [6]:
model = ResNet50()
bc_model_weights = sc.broadcast(model.get_weights())

2024-09-19 18:10:34.188324: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB-H, pci bus id: 0000:34:00.0, compute capability: 7.0


Load the data and save the datasets to one Parquet file.

In [7]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [8]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [9]:
import os
files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) for f in filenames if os.path.splitext(f)[1] == '.jpg']
files = files[:2048]
len(files)

2048

In [10]:
print(data_dir)

/rishic/.keras/datasets/flower_photos


In [11]:
image_data = []
for file in files:
    img = Image.open(file)
    img = img.resize([224, 224])
    data = np.asarray(img, dtype="float32").reshape([224*224*3])

    image_data.append({"data": data})

pandas_df = pd.DataFrame(image_data, columns=['data'])
pandas_df.to_parquet(file_name)
# os.makedirs(dbfs_file_path)
# shutil.copyfile(file_name, dbfs_file_path+file_name)

### Save Model


In [12]:
subprocess.call("rm -rf resnet50_model".split())

0

In [13]:
model.save('resnet50_model')

INFO:tensorflow:Assets written to: resnet50_model/assets


INFO:tensorflow:Assets written to: resnet50_model/assets


### Load the data into Spark DataFrames

In [14]:
from pyspark.sql.types import *
df = spark.read.parquet(file_name)
print(df.count())

2048


In [15]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [16]:
assert len(df.head()) > 0, "`df` should not be empty" # This line will fail if the vectorized reader runs out of memory

### Model inference via pandas UDF

In [17]:
def parse_image(image_data):
    image = tf.image.convert_image_dtype(
        image_data, dtype=tf.float32) * (2. / 255) - 1
    image = tf.reshape(image, [224, 224, 3])
    return image

In [18]:
@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
def predict_batch_udf(image_batch_iter):

    # Enable GPU memory growth to avoid CUDA OOM
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    batch_size = 64
    model = ResNet50(weights=None)
    model.set_weights(bc_model_weights.value)
    for image_batch in image_batch_iter:
        images = np.vstack(image_batch)
        dataset = tf.data.Dataset.from_tensor_slices(images)
        dataset = dataset.map(parse_image, num_parallel_calls=8).prefetch(
            5000).batch(batch_size)
        preds = model.predict(dataset)
        yield pd.Series(list(preds))

/rishic/anaconda3/envs/spark-rapids-examples/lib/python3.11/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [19]:
%%time
predictions_df = df.select(predict_batch_udf(col("data")).alias("prediction"))
predictions_df.show(truncate=120)

2024-09-19 18:11:15.650739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:11:15.650779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 18:11:15.652313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 18:11:15.658996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 18:11:16.263639: W tensorflow/comp

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[4.7789E-5, 3.036924E-4, 7.026333E-5, 1.7727503E-4, 4.568335E-5, 1.9182995E-4, 9.988362E-6, 7.264467E-5, 1.4183699E-5...|
|[1.2047249E-5, 2.4665342E-4, 2.3863076E-4, 1.930134E-4, 1.5375498E-4, 4.5055378E-5, 2.2207623E-5, 3.7911965E-4, 1.570...|
|[1.3302322E-4, 2.6962237E-4, 5.5178854E-5, 9.990185E-5, 4.761864E-5, 4.404573E-4, 6.805496E-6, 3.4860044E-5, 1.267329...|
|[1.519048E-5, 2.9297205E-4, 1.1742505E-4, 8.6468724E-5, 7.02726E-5, 7.291901E-5, 1.1590414E-5, 2.9054165E-4, 1.735722...|
|[1.1257287E-4, 2.5857842E-4, 5.529734E-5, 1.04460676E-4, 4.6776797E-5, 3.4146337E-4, 6.6849507E-6, 3.6820882E-5, 1.15...|
|[9.9512894E-5, 

In [20]:
%%time
predictions_df.write.mode("overwrite").parquet(output_file_path)

2024-09-19 18:11:38.904599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:11:38.904601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:11:38.904601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:11:38.904645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 18:11:38.904645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory f

CPU times: user 21.9 ms, sys: 12.5 ms, total: 34.4 ms
Wall time: 29.9 s


### Model inference using Spark DL API

In [21]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct, col
from pyspark.sql.types import ArrayType, FloatType

In [22]:
def predict_batch_fn():
    import tensorflow as tf
    from tensorflow.keras.applications.resnet50 import ResNet50

    # Enable GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = ResNet50()
    def predict(inputs):
        inputs = inputs * (2. / 255) - 1
        return model.predict(inputs)
    return predict

In [23]:
classify = predict_batch_udf(predict_batch_fn,
                             input_tensor_shapes=[[224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [24]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [25]:
df = spark.read.parquet("image_data.parquet")

In [26]:
%%time
# first pass caches model/fn
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

2024-09-19 18:12:18.423116: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[4.7789E-5, 3.036924E-4, 7.026333E-5, 1.7727503E-4, 4.568335E-5, 1.9182995E-4, 9.988362E-6, 7.264467E-5, 1.4183699E-5...|
|[1.2047249E-5, 2.4665342E-4, 2.3863076E-4, 1.930134E-4, 1.5375498E-4, 4.5055378E-5, 2.2207623E-5, 3.7911965E-4, 1.570...|
|[1.3302322E-4, 2.6962237E-4, 5.5178854E-5, 9.990185E-5, 4.761864E-5, 4.404573E-4, 6.805496E-6, 3.4860044E-5, 1.267329...|
|[1.519048E-5, 2.9297205E-4, 1.1742505E-4, 8.6468724E-5, 7.02726E-5, 7.291901E-5, 1.1590414E-5, 2.9054165E-4, 1.735722...|
|[1.1257287E-4, 2.5857842E-4, 5.529734E-5, 1.04460676E-4, 4.6776797E-5, 3.4146337E-4, 6.6849507E-6, 3.6820882E-5, 1.15...|
|[9.9512894E-5, 

1/1 [==============================] - 3s 3s/step


In [27]:
%%time
predictions = df.select(classify("data").alias("prediction"))
predictions.show(truncate=120)

2024-09-19 18:12:26.457803: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[4.7789E-5, 3.036924E-4, 7.026333E-5, 1.7727503E-4, 4.568335E-5, 1.9182995E-4, 9.988362E-6, 7.264467E-5, 1.4183699E-5...|
|[1.2047249E-5, 2.4665342E-4, 2.3863076E-4, 1.930134E-4, 1.5375498E-4, 4.5055378E-5, 2.2207623E-5, 3.7911965E-4, 1.570...|
|[1.3302322E-4, 2.6962237E-4, 5.5178854E-5, 9.990185E-5, 4.761864E-5, 4.404573E-4, 6.805496E-6, 3.4860044E-5, 1.267329...|
|[1.519048E-5, 2.9297205E-4, 1.1742505E-4, 8.6468724E-5, 7.02726E-5, 7.291901E-5, 1.1590414E-5, 2.9054165E-4, 1.735722...|
|[1.1257287E-4, 2.5857842E-4, 5.529734E-5, 1.04460676E-4, 4.6776797E-5, 3.4146337E-4, 6.6849507E-6, 3.6820882E-5, 1.15...|
|[9.9512894E-5, 

1/1 [==============================] - 3s 3s/step


In [28]:
%%time
predictions = df.select(classify(col("data")).alias("prediction"))
predictions.write.mode("overwrite").parquet(output_file_path + "_1")

2024-09-19 18:12:29.165736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:12:29.165766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 18:12:29.167063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 18:12:29.173618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 18:12:29.194920: E external/local_

CPU times: user 52.3 ms, sys: 13 ms, total: 65.3 ms
Wall time: 29.5 s


2/2 [==============================] - 1s 1s/step


### Using Triton Inference Server

Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [29]:
import os
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, struct
from pyspark.sql.types import ArrayType, FloatType

In [30]:
%%bash
# copy model to expected layout for Triton
rm -rf models
mkdir -p models/resnet50/1
cp -r resnet50_model models/resnet50/1/model.savedmodel

# add config.pbtxt
cp models_config/resnet50/config.pbtxt models/resnet50/config.pbtxt

#### Start Triton Server on each executor

In [31]:
num_executors = 1
triton_models_dir = "{}/models".format(os.getcwd())
nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:23.04-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="512M",
            volumes={triton_models_dir: {"bind": "/models", "mode": "ro"}}
        )
        print(">>>> starting triton: {}".format(container.short_id))

        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)

    return [True]

nodeRDD.barrier().mapPartitions(start_triton).collect()

>>>> starting triton: 93fd0e583119                                  (0 + 1) / 1]


[True]

#### Run inference

In [32]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool8),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            inputs = inputs * (2. / 255) - 1  # add normalization
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

In [33]:
from functools import partial

classify = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="resnet50"),
                             input_tensor_shapes=[[224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [34]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [35]:
df = spark.read.parquet("image_data.parquet")

In [36]:
%%time
# first pass caches model/fn
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)


+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[4.778903E-5, 3.0369128E-4, 7.026332E-5, 1.7727482E-4, 4.568349E-5, 1.9182982E-4, 9.988365E-6, 7.264466E-5, 1.4183701...|
|[1.204735E-5, 2.466547E-4, 2.3863222E-4, 1.9301496E-4, 1.5375607E-4, 4.5055716E-5, 2.2207836E-5, 3.7912003E-4, 1.5702...|
|[1.3302316E-4, 2.6962277E-4, 5.5178938E-5, 9.9902085E-5, 4.7618734E-5, 4.4045778E-4, 6.805516E-6, 3.486011E-5, 1.2673...|
|[1.5190468E-5, 2.9297193E-4, 1.17425334E-4, 8.646906E-5, 7.0272734E-5, 7.291894E-5, 1.1590415E-5, 2.905404E-4, 1.7357...|
|[1.1257278E-4, 2.5857845E-4, 5.5297292E-5, 1.0446065E-4, 4.6776808E-5, 3.4146375E-4, 6.684932E-6, 3.682082E-5, 1.1539...|
|[9.951307E-5, 3

In [37]:
%%time
predictions = df.select(classify("data").alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[4.7788984E-5, 3.0369213E-4, 7.026345E-5, 1.7727505E-4, 4.56834E-5, 1.9182981E-4, 9.988383E-6, 7.264468E-5, 1.4183702...|
|[1.2047228E-5, 2.4665325E-4, 2.3863037E-4, 1.9301346E-4, 1.537548E-4, 4.5055393E-5, 2.2207578E-5, 3.7911886E-4, 1.570...|
|[1.3302319E-4, 2.6962243E-4, 5.5178865E-5, 9.9901925E-5, 4.7618672E-5, 4.4045722E-4, 6.805494E-6, 3.4860033E-5, 1.267...|
|[1.5190472E-5, 2.9297202E-4, 1.1742514E-4, 8.646884E-5, 7.0272596E-5, 7.2919065E-5, 1.1590418E-5, 2.905416E-4, 1.7357...|
|[1.1257286E-4, 2.585784E-4, 5.5297358E-5, 1.0446073E-4, 4.677684E-5, 3.414637E-4, 6.684947E-6, 3.6820864E-5, 1.153909...|
|[9.951301E-5, 3

In [38]:
%%time
predictions = df.select(classify(col("data")).alias("prediction"))
predictions.write.mode("overwrite").parquet(output_file_path + "_2")

/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
/tmp/ipykernel_3665537/4086744192.py:6: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)


CPU times: user 18.1 ms, sys: 5.55 ms, total: 23.6 ms
Wall time: 27.9 s


#### Stop Triton Server on each executor

In [39]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.barrier().mapPartitions(stop_triton).collect()

>>>> stopping containers: ['93fd0e583119']


[True]

In [40]:
spark.stop()